In [8]:
%load_ext Cython

In [10]:
import copy
import igraph
import json
import cairo
import pandas as pd
import sys
import numpy as np
import string
import random
import matplotlib.pyplot as plt 
import collections
#from scipy.special import factorial
%matplotlib inline

In [11]:

def make_board(n=7):
    board = set(range(1,n+1))
    return board

class GameNode:
    # def __init__(self, movein_player=2, board=make_board(), collected_points=[[],[],[]], sums=[0,0,0], moves=[], level=0):
    def __init__(self, movein_player=2, board=make_board(), sums=None, level=0):
        if not sums:
            sums = [sum(board), 0, 0]
            
        self.movein_player = movein_player
        self.moveout_player = 3-movein_player
        self.board = copy.copy(board)
        #self.collected_points = copy.deepcopy(collected_points)
        self.sums = [sums[0], sums[1], sums[2]]
        self.movein = []  # this is the move that got us to this node
        self.moves = []  # these are the possible moves after this node
        self.children = []
        self.level = level
        self.result = None
        self.sgvalue = None
        self.vertexid = None
        self.graphed = False
        
    def spawn(self):
        # node= GameNode(self.movein_player, self.board, self.collected_points, self.sums)
        node= GameNode(self.movein_player, self.board, self.sums)
        node.result = self.result
        node.sgvalue = self.sgvalue
        return node
        
    def show2(self, recursive=False):
        tab = "  "*self.level
        print(tab+"P"+str(self.moveout_player)+":"+str(self.board)+":"+str(self.sums)+" "+str(self.win()))
        if recursive:
            for child in self.children:
                child.show(recursive)
                
    def show(self, recursive=False):
        tab = "  "*self.level
        print(tab+"P"+str(self.movein_player)+""+str(self.movein)+":"+str(self.board)+":"+str(self.sums)+" "+str(self.result))
        if recursive:
            for child in self.children:
                child.show(recursive)
        
    def setmoves(self, trimming=True):
        self.moves = self.possible_moves(trimming)
        
    def push_move(self, move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level += 1
        for m in move:
            self.board.discard(m)
            self.sums[self.movein_player] += m
            self.sums[0] -= m

    def pop_move(self, move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level -= 1
        for m in move:
            self.board.add(m)
            self.sums[self.moveout_player] -= m
            self.sums[0] += m
        
    def make_key(self):
        return str(self.movein_player)+":"+str(self.sums[1])+'/'+str(self.sums[2])+":"+str(self.board)
    
    def make_move(self, move):
        node = self.spawn()
        node.movein_player = self.moveout_player
        node.moveout_player = self.movein_player
        
        #node.collected_points[node.movein_player].append(move)
        
        for pick in move:
            node.sums[node.movein_player] += pick
            node.sums[0] -= pick
            node.board.discard(pick)

        node.movein = move
        node.level = self.level + 1
    
        self.children.append(node)
        
        return node

    def generate_move(self, maxplay, sum_move=0, move=[], picks=None, trimming=True):
        if picks == None:
            picks = self.board

        if move:
            maxpick = max(move)
        else:
            maxpick = 0
            
        len_move_plus_1 = len(move)+1
        len_picks = len(picks)

        for pick in picks:
            if pick < maxpick:
                # only emit the moves in increasing order 
                # (i.e don't emit both 2,3 and 3,2 if you are behind by 4, just emit 2,3)
                if trimming:
                    continue

            if pick in move:
                # can't pick something you've already picked
                continue

            move.append(pick)
            if len_move_plus_1 == len_picks:
                yield list(move)
            elif sum_move+pick >= maxplay:
                yield list(move)
            else:
                for m in self.generate_move(maxplay, sum_move+pick, move, picks, trimming):
                    yield m
            move.pop()

    def scoreP1(self):
        sA = self.sums[1]
        sB = self.sums[2]
        return sA-sB
    
    def win(self):
        s = self.score()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    def winP1(self):
        s = self.scoreP1()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    def score(self):
        sA = self.sums[self.movein_player]
        sB = self.sums[self.moveout_player]
        return sA-sB
    
    
    
    def possible_moves(self, trimming):
        # A is the current player
        # B is the opponent player
        sA = self.sums[self.moveout_player]
        sB = self.sums[self.movein_player]

        maxplay = sB - sA
        #assert maxplay >= 0

        moves = list(self.generate_move(maxplay, trimming=trimming))
        return moves
      
        

## playing the game - heuristics
we need to play the game with different heuristics to discover which is most interesting

In [2]:
class Strategy:
    def __init__(self):
        pass
    
    def PickMove(self, node):
        return []
    
class RandomStrategy(Strategy):
    def __init__(self):
        pass
    
    def PickMove(self, node):
        moves = node.possible_moves()
        move = random.choice(moves)
        return move
    
class MaximizeScoreStrategy(Strategy):
    def __init__(self):
        pass
    
    def _DoPickMove(self, moves):
        movesum = [sum(move) for move in moves]
        target = max(movesum)
        indices = [i for i, x in enumerate(movesum) if x == target]
        index = random.choice(indices)
        return moves[index]
    
    def PickMove(self, node):
        moves = node.possible_moves()
        return self._DoPickMove(moves)


class MinimizeScoreStrategy(Strategy):
    def __init__(self):
        pass
    
    def _DoPickMove(self, moves):
        movesum = [sum(move) for move in moves]
        target = min(movesum)
        indices = [i for i, x in enumerate(movesum) if x == target]
        index = random.choice(indices)
        return moves[index]
    
    def PickMove(self, node):
        moves = node.possible_moves()
        return self._DoPickMove(moves)
    
class UseMostPiecesStrategy(Strategy):
    def __init__(self):
        pass
    
    def _DoPickMove(self, moves):
        movesum = [len(move) for move in moves]
        target = max(movesum)
        indices = [i for i, x in enumerate(movesum) if x == target]
        index = random.choice(indices)
        return moves[index]
    
    def PickMove(self, node):
        moves = node.possible_moves()
        return self._DoPickMove(moves)

class UseMostPiecesThenMaximizeStrategy(Strategy):
    def __init__(self):
        pass
    
    def _DoPickMove(self, moves):
        movelen = [len(move) for move in moves]
        movesum = [sum(move) for move in moves]
        target = max(movelen)
        indices = [i for i, x in enumerate(movesum) if x == target]
        if len(indices) == 1:
            return moves[indices[0]]
        else:
            movesummax =0
        
            for index in indices:
                movesummax = max(movesummax, movesum[index])
                
        index = random.choice(indices)
        return moves[index]
    
    def PickMove(self, node):
        moves = node.possible_moves()
        return self._DoPickMove(moves)
    
class UseLeastPiecesStrategy(Strategy):
    def __init__(self):
        pass
    
    def _DoPickMove(self, moves):
        movesum = [len(move) for move in moves]
        target = min(movesum)
        indices = [i for i, x in enumerate(movesum) if x == target]
        index = random.choice(indices)
        return moves[index]
    
    def PickMove(self, node):
        moves = node.possible_moves()
        return self._DoPickMove(moves)


    
class GamePlayer:
    def __init__(self, player1strategy, player2strategy, n=7):
        self.n = n
        self.strategies = [None, player1strategy, player2strategy]
    
    def Play(self):
        game = GameNode(board= make_board(self.n))
        while game.board:
            player = game.moveout_player
            move = self.strategies[player].PickMove(game)
            game.push_move(move)
        return game.winP1()
    
    def Experiment(self, n=10000):
        data = pd.Series([self.Play() for i in xrange(n)])
        return data.value_counts()
    

## Playing Against a Random Strategy
What is the best thing to do against a random strategy?

First, lets see what is the best heursitic when playing against a Random P2

In [ ]:
GamePlayer(MinimizeScoreStrategy(), RandomStrategy(), 9).Experiment()

In [ ]:
GamePlayer(RandomStrategy(), RandomStrategy(), 9).Experiment()

In [ ]:
GamePlayer(MaximizeScoreStrategy(), RandomStrategy(), 9).Experiment()

So, P1 prefers Maximize if playing against Random.  We can now assume that P2 switches to a new strategy since P1 is going to play Maximize. 


In [ ]:
GamePlayer(MaximizeScoreStrategy(), MinimizeScoreStrategy(), 9).Experiment()

In [ ]:
GamePlayer(MaximizeScoreStrategy(), RandomStrategy(), 9).Experiment()

In [ ]:
GamePlayer(MaximizeScoreStrategy(), MaximizeScoreStrategy(), 9).Experiment()

P2 thus moves to Maximize to match P1, and now P2 is always winning

In [ ]:
GamePlayer(MaximizeScoreStrategy(), MaximizeScoreStrategy(), 9).Experiment()

In [ ]:
GamePlayer(RandomStrategy(), MaximizeScoreStrategy(), 9).Experiment()

In [ ]:
GamePlayer(MinimizeScoreStrategy(), MaximizeScoreStrategy(), 9).Experiment()

In [ ]:
GamePlayer(UseMostPiecesStrategy(), MaximizeScoreStrategy(), 9).Experiment()

In [ ]:
GamePlayer(UseLeastPiecesStrategy(), MaximizeScoreStrategy(), 9).Experiment()

P2 seems to do very well with the maximize strategy with 1...9.  What about other games?

In [ ]:
df = pd.DataFrame(index=[4,5,6,7,8,9,10], columns=[-1,0,1])
for i in df.index:
    df.loc[i] = GamePlayer(MinimizeScoreStrategy(), MaximizeScoreStrategy(), i).Experiment()
df

In [ ]:
s1 = MinimizeScoreStrategy()
s2 = MinimizeScoreStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [ ]:
s1 = MinimizeScoreStrategy()
s2 = RandomStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [ ]:
s1 = MinimizeScoreStrategy()
s2 = MaximizeScoreStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [ ]:
s1 = MaximizeScoreStrategy()
s2 = MinimizeScoreStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [ ]:
s1 = MaximizeScoreStrategy()
s2 = MaximizeScoreStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [ ]:
s1 = RandomStrategy()
s2 = RandomStrategy()
gp = GamePlayer(s1, s2, 9)
gp.Experiment()

In [24]:
class FastTreeExplore:
    def __init__(self, n=4, trimming=True):
        self.n = n
        self.trimming = trimming
        self.root = GameNode(board=make_board(n))
    
    def num_nodes(self):
        def _explore(node):
            n = 1
            for move in node.possible_moves(self.trimming):
                node.push_move(move)
                n += _explore(node)
                node.pop_move(move)
            return n
        return _explore(self.root)
    
    def avg_branching_factor(self):
        d = {'count': 0, 'sumbranch': 0, 'maxbranch':0, 'avgbranch':0}
        def _explore(node):
            moves = node.possible_moves(self.trimming)
            len_moves = len(moves)
            if len_moves == 0:
                return 
            else:
                d['count'] += 1
                d['sumbranch'] += len_moves
                d['maxbranch'] = max(d['maxbranch'], len_moves)
                for move in moves:
                    node.push_move(move)
                    _explore(node)
                    node.pop_move(move)
        _explore(self.root)
        d['avgbranch'] = float(d['sumbranch'])/d['count']
        return d
    
    def avg_branching_factor_by_level(self):
        d = [{'count': 0, 'sumbranch': 0, 'maxbranch':0, 'avgbranch':0} for i in xrange(self.n)]
        
        def _explore(node):
            moves = node.possible_moves(self.trimming)
            len_moves = len(moves)
            if len_moves == 0:
                return 
            else:
                level = d[node.level]
                level['count'] += 1
                level['sumbranch'] += len_moves
                level['maxbranch'] = max(level['maxbranch'], len_moves)
                for move in moves:
                    node.push_move(move)
                    _explore(node)
                    node.pop_move(move)
        _explore(self.root)
        for level in d:
            level['avgbranch'] = float(level['sumbranch'])/level['count']
            
        return d
       
    def random_play(self):
        node = self.root
        while True:
            moves = node.possible_moves(self.trimming)
            if not moves:
                # gameover
                break
            else:
                # more moves
                move = random.choice(moves)
                node.push_move(move)
                
        # return result from perspective of P1
        result = node.winP1()
        return result
   
    def count_win_lose_draw(self, startnode=None):
        wld = {'WIN':0,'LOSE':0,'TIE':0,'TOTAL':0}
               
        def _explore(node):
            moves = node.possible_moves(self.trimming)
            if not moves:
                win = node.winP1()
                if win == 1:
                    wld['WIN'] += 1
                elif win == -1:
                    wld['LOSE'] += 1
                elif win == 0:
                    wld['TIE'] += 1
                wld['TOTAL'] += 1
            else:
                for move in moves:
                    node.push_move(move)
                    _explore(node)
                    node.pop_move(move)
                    
        if startnode:
            _explore(startnode)
        else:
            _explore(self.root)
        
        wld['WIN%']  = float(wld['WIN'])/wld['TOTAL']
        wld['LOSE%']  = float(wld['LOSE'])/wld['TOTAL']
        wld['TIE%']  = float(wld['TIE'])/wld['TOTAL']
        
        return wld
 
    def count_win_lose_draw_cached(self, firstmove=None):
        wld = {'WIN':0,'LOSE':0,'TIE':0,'TOTAL':0}
        cache = {}
               
        def _explore(node):
            key = node.make_key()
            
            # we've been here before, return the cached value
            if cache.has_key(key):
                return cache[key]
            
            result = np.array((0,0,0,0))
            moves = node.possible_moves(self.trimming)
            if not moves:
                win = node.winP1()
                if win == 1:
                    return np.array((1,0,0,1))
                elif win == -1:
                    return np.array((0,1,0,1))
                elif win == 0:
                    return np.array((0,0,1,1))
            else:
                for move in moves:
                    node.push_move(move)
                    result += _explore(node)
                    node.pop_move(move)
            cache[key] = result
            return result
        
        node = self.root
        if firstmove:
            node.push_move(firstmove)
        
        result = _explore(node)
        
        wld['WIN'] = result[0]
        wld['LOSE'] = result[1]
        wld['TIE'] = result[2]
        wld['TOTAL'] = result[3]
        wld['WIN%']  = float(wld['WIN'])/wld['TOTAL']
        wld['LOSE%']  = float(wld['LOSE'])/wld['TOTAL']
        wld['TIE%']  = float(wld['TIE'])/wld['TOTAL']
        
        return wld
    
    
    def count_win_lose_draw_cached_fast(self, firstmove=None):
        wld = {'WIN':0,'LOSE':0,'TIE':0,'TOTAL':0}
        cache = {}
        factarray = [factorial(i, exact=True) for i in xrange(self.n+1)]
               
        def _explore(node):
            key = node.make_key()
            
            # we've been here before, return the cached value
            if cache.has_key(key):
                return cache[key]
            
            result = np.array((0,0,0,0))
            moves = node.possible_moves(trimming=True)
            if not moves:
                win = node.winP1()
                if win == 1:
                    return np.array((1,0,0,1))
                elif win == -1:
                    return np.array((0,1,0,1))
                elif win == 0:
                    return np.array((0,0,1,1))
            else:
                for move in moves:
                    if move[-1] < node.score():
                        extra = factarray[len(move)]
                    else:
                        extra = factarray[len(move)-1]
                    #print("%d %s" % (extra, str(move)))


                        
                    node.push_move(move)
                    result += _explore(node)*extra
                    node.pop_move(move)
            cache[key] = result
            return result
        
        node = self.root
        if firstmove:
            node.push_move(firstmove)
        
        result = _explore(node)
        
        wld['WIN'] = result[0]
        wld['LOSE'] = result[1]
        wld['TIE'] = result[2]
        wld['TOTAL'] = result[3]
        wld['WIN%']  = float(wld['WIN'])/wld['TOTAL']
        wld['LOSE%']  = float(wld['LOSE'])/wld['TOTAL']
        wld['TIE%']  = float(wld['TIE'])/wld['TOTAL']
        
        return wld
    
    def find_duplicates(self):
        d = collections.defaultdict(int)
        
        def _explore(node):
            key = node.make_key()
            d[key] += 1
            d['TOTAL'] += 1
            
            moves = node.possible_moves(self.trimming)
            if not moves:
                return 
            else:
                for move in moves:
                    node.push_move(move)
                    _explore(node)
                    node.pop_move(move)
        _explore(self.root)
        
        return d
    
    def unique_states(self):
        cache = collections.defaultdict(int)
        
        def _explore(node):
            key = node.make_key()
            if cache.has_key(key):
                return
            else:
                cache[key] += 1
                cache['TOTAL'] += 1
            
            moves = node.possible_moves(self.trimming)
            if not moves:
                return 
            else:
                for move in moves:
                    node.push_move(move)
                    _explore(node)
                    node.pop_move(move)
        _explore(self.root)
        
        return cache['TOTAL']
    
    def depth(self):
        d = {'min':99999, 'max':0}
        def _explore(node, depth):
            moves = node.possible_moves(self.trimming)
            if len(moves) == 0:
                d['min'] = min(d['min'], depth)
                d['max'] = max(d['max'], depth)
            else:
                for move in moves:
                    node.push_move(move)
                    _explore(node, depth+1)
                    node.pop_move(move)
        _explore(self.root, 0)
        return d
    
    def mindepth_bfs(self):
        queue = [self.root]
        iqueue = 0
        mindepth = 99999
        while iqueue < len(queue):
            node = queue[iqueue]
            iqueue += 1 # this avoids popping, we can just leave nodes in the queue but ignore them
            
            # only expand the node if we might find a better solution
            if node.level >= mindepth:
                continue
                
            # expand the node
            node.setmoves()
            
            for move in node.moves:
                child = node.make_move(move)
                if not child.board:
                    # found a terminal node, check its depth
                    mindepth = min(mindepth, child.level)
                else:
                    # not a terminal node, but only expand if its shallower than our shortest terminal game
                    if child.level < mindepth:
                        queue.append(child)
            
        return mindepth

    
    def mindepth_id(self):
        # use iterative deepening
        def _explore(node, depth, maxdepth):
            #if depth >= maxdepth:
            #    return maxdepth+1
            
            if node.sums[0] == 0:
                # found a game that ends at maxdepth, we are done
                return depth           
            elif depth+1 <= maxdepth:
                # haven't reached the maxdepth yet, keep looking
                moves = node.possible_moves(self.trimming)
                for move in moves:
                    node.push_move(move)
                    best = _explore(node, depth+1, maxdepth)
                    node.pop_move(move)
                    
                    if best <= maxdepth:
                        return best
                # if here, we didn't find anything
                return maxdepth+1
            else:
                # fail case, truncating
                return maxdepth+1
        
        for iterdepth in xrange(1, self.n+1):
            v = _explore(self.root, 0, iterdepth)
            if v <= iterdepth:
                return v
        
        return -1
    
    def mindepth(self):
        d = {'min':99999}
        def _explore(node, depth):
            moves = node.possible_moves(self.trimming)
            if not moves:
                d['min'] = min(d['min'], depth)
            elif d['min'] == depth+1:
                # already found a min, don't search deeper
                return 
            else:
                # haven't reached the min yet, keep looking
                for move in moves:
                    node.push_move(move)
                    _explore(node, depth+1)
                    node.pop_move(move)
                    
                    # if we found an end game we don't have to search the rest
                    if d['min'] == depth+1:
                        break
                        
        _explore(self.root, 0)
        return d['min']
 
    
    def num_terminals(self):
        def _explore(node):
            n = 0
            moves = node.possible_moves(self.trimming)
            if len(moves) == 0:
                n+=1
            else:
                for move in moves:
                    node.push_move(move)
                    n += _explore(node)
                    node.pop_move(move)
            return n
        return _explore(self.root)
    
        
    def num_terminals_cached(self):
        cache = collections.defaultdict(int)
        def _explore(node):
            n=0
            moves = node.possible_moves(self.trimming)
            if len(moves) == 0:
                n+=1
            else:
                for move in moves:
                    node.push_move(move)
                    key = node.make_key()
                    if cache.has_key(key):
                        n += cache[key]
                    else:
                        value = _explore(node)
                        n += value
                        cache[key] = value
                    node.pop_move(move)
            return n
        return _explore(self.root)
    
    def negamax(self):
        # negamax code ported from https://en.wikipedia.org/wiki/Negamax
        def _explore(node, depth, color):
            #node.show2()
            
            if depth == 0:
                return color * node.winP1()
            moves = node.possible_moves(self.trimming)
            if not moves:
                return color * node.winP1()

            bestValue = -100
            for move in moves:
                node.push_move(move)
                val = -_explore(node, depth-1, -color)
                node.pop_move(move)
                bestValue = max( bestValue, val )
            return bestValue
                  
        return _explore(self.root, self.n+1, 1)
    
    def negamax_cached(self):
        # negamax code ported from https://en.wikipedia.org/wiki/Negamax
        cache = collections.defaultdict(int)
        
        def _explore(node, depth, color):
            #node.show2()
            key = node.make_key()
            if cache.has_key(key):
                return cache[key]
            
            if depth == 0:
                cache[key] = color * node.winP1()
                return color * node.winP1()
            moves = node.possible_moves(self.trimming)
            if not moves:
                cache[key] = color * node.winP1()
                return color * node.winP1()

            bestValue = -100
            for move in moves:
                node.push_move(move)
                val = -_explore(node, depth-1, -color)
                node.pop_move(move)
                bestValue = max( bestValue, val )
            cache[key] = bestValue
            return bestValue
        
        return _explore(self.root, self.n+1, 1)
    
    
            

## Win Lose Draw ratios
These are the actual win lose and draw ratios, not sampled from random play

In [27]:
FastTreeExplore(4, trimming=False).count_win_lose_draw()

{'LOSE': 8,
 'LOSE%': 0.3333333333333333,
 'TIE': 8,
 'TIE%': 0.3333333333333333,
 'TOTAL': 24,
 'WIN': 8,
 'WIN%': 0.3333333333333333}

In [46]:
n = 9
fte = FastTreeExplore(n, trimming=True)
fte.root.push_move([3])
for move in fte.root.possible_moves(trimming=True):
    fte.root.push_move(move)
    print((move, fte.count_win_lose_draw()))
    fte.root.pop_move(move)


([1, 2], {'WIN': 265, 'LOSE': 308, 'TIE%': 0.0, 'LOSE%': 0.537521815008726, 'TIE': 0, 'TOTAL': 573, 'WIN%': 0.462478184991274})
([1, 4], {'WIN': 375, 'LOSE': 256, 'TIE%': 0.0, 'LOSE%': 0.40570522979397783, 'TIE': 0, 'TOTAL': 631, 'WIN%': 0.5942947702060222})
([1, 5], {'WIN': 322, 'LOSE': 299, 'TIE%': 0.0, 'LOSE%': 0.48148148148148145, 'TIE': 0, 'TOTAL': 621, 'WIN%': 0.5185185185185185})
([1, 6], {'WIN': 332, 'LOSE': 275, 'TIE%': 0.0, 'LOSE%': 0.45304777594728174, 'TIE': 0, 'TOTAL': 607, 'WIN%': 0.5469522240527183})
([1, 7], {'WIN': 302, 'LOSE': 270, 'TIE%': 0.0, 'LOSE%': 0.47202797202797203, 'TIE': 0, 'TOTAL': 572, 'WIN%': 0.527972027972028})
([1, 8], {'WIN': 296, 'LOSE': 238, 'TIE%': 0.0, 'LOSE%': 0.44569288389513106, 'TIE': 0, 'TOTAL': 534, 'WIN%': 0.5543071161048689})
([1, 9], {'WIN': 238, 'LOSE': 256, 'TIE%': 0.0, 'LOSE%': 0.5182186234817814, 'TIE': 0, 'TOTAL': 494, 'WIN%': 0.4817813765182186})
([2, 4], {'WIN': 458, 'LOSE': 203, 'TIE%': 0.0, 'LOSE%': 0.3071104387291982, 'TIE': 0, '

In [164]:
%timeit FastTreeExplore(8, trimming=False).count_win_lose_draw_cached()

10 loops, best of 3: 59.2 ms per loop


In [130]:
FastTreeExplore(5, trimming=False).count_win_lose_draw_cached_fast()

{'LOSE': 64,
 'LOSE%': 0.47761194029850745,
 'TIE': 0,
 'TIE%': 0.0,
 'TOTAL': 134,
 'WIN': 70,
 'WIN%': 0.5223880597014925}

In [71]:
t = FastTreeExplore(4)
t.root.push_move([4])
print(t.root.possible_moves(trimming=False))
print(t.root.possible_moves(trimming=True))


[[1, 2, 3], [1, 3], [2, 1, 3], [2, 3], [3, 1], [3, 2]]
[[1, 2, 3], [1, 3], [2, 3]]


In [39]:
r = range(3,21)
df = pd.DataFrame(columns=['WIN', 'WIN%', 'LOSE', 'LOSE%', 'TIE', 'TIE%', 'TOTAL'], index=r)
for n in r:
    %time v = FastTreeExplore(n, trimming=False).count_win_lose_draw_cached()
    print((n,v))
    df.loc[n] = v
win_lose_draw_table = df
win_lose_draw_table

CPU times: user 206 µs, sys: 52 µs, total: 258 µs
Wall time: 232 µs
(3, {'WIN': 1, 'LOSE': 1, 'TIE%': 0.6666666666666666, 'LOSE%': 0.16666666666666666, 'TIE': 4, 'TOTAL': 6, 'WIN%': 0.16666666666666666})
CPU times: user 585 µs, sys: 151 µs, total: 736 µs
Wall time: 630 µs
(4, {'WIN': 8, 'LOSE': 8, 'TIE%': 0.3333333333333333, 'LOSE%': 0.3333333333333333, 'TIE': 8, 'TOTAL': 24, 'WIN%': 0.3333333333333333})
CPU times: user 1.72 ms, sys: 527 µs, total: 2.25 ms
Wall time: 1.86 ms
(5, {'WIN': 60, 'LOSE': 60, 'TIE%': 0.0, 'LOSE%': 0.5, 'TIE': 0, 'TOTAL': 120, 'WIN%': 0.5})
CPU times: user 5.92 ms, sys: 2.17 ms, total: 8.09 ms
Wall time: 6.47 ms
(6, {'WIN': 360, 'LOSE': 360, 'TIE%': 0.0, 'LOSE%': 0.5, 'TIE': 0, 'TOTAL': 720, 'WIN%': 0.5})
CPU times: user 20.1 ms, sys: 6.98 ms, total: 27.1 ms
Wall time: 22.2 ms
(7, {'WIN': 1944, 'LOSE': 1944, 'TIE%': 0.22857142857142856, 'LOSE%': 0.38571428571428573, 'TIE': 1152, 'TOTAL': 5040, 'WIN%': 0.38571428571428573})
CPU times: user 65.9 ms, sys: 8.34 ms

KeyboardInterrupt: 

In [ ]:
%time FastTreeExplore(9).negamax_cached()

In [ ]:
t = TreeBuilder(5)
t.build(cache=True)
t.calc_results()
-t.root.result

## does the first move matter?

In [ ]:
n=7
df = pd.DataFrame(columns=['WIN', 'WIN%', 'LOSE', 'LOSE%', 'TIE', 'TIE%', 'TOTAL'], index=range(0,n))
for fm in xrange(0,n+1):
    if fm == 0:
        v = FastTreeExplore(n, trimming=False).count_win_lose_draw_cached()
    else:
        v = FastTreeExplore(n, trimming=False).count_win_lose_draw_cached([fm])
    #print(([fm],v))
    df.loc[fm] = v
win_lose_draw_table = df
win_lose_draw_table

In [ ]:
(win_lose_draw_table - win_lose_draw_table.loc[0])*100


In [ ]:
print(win_lose_draw_table.to_latex())

## Random Play

What are the odds of losing, winning, or tying given random play?

In [41]:
df = pd.DataFrame(columns=range(8,9), index=[-1,0,1])
n = 10000
for c in df.columns:
    df[c] = pd.Series([FastTreeExplore(c, trimming=False).random_play() for i in xrange(n)]).value_counts()
df = df.fillna(0)
df = df/n
random_win_lose_draw = df
random_win_lose_draw

,8
-1,0.3875
0,0.1951
1,0.4174


In [34]:
[FastTreeExplore(7).random_play() for i in xrange(10)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
FastTreeExplore(7).random_play()


1

In [10]:
FastTreeExplore(4).negamax()

0

In [ ]:
t = FastTreeExplore(8)
d = t.find_duplicates()
print((len(d)-1, d['TOTAL']))
t.unique_states()

In [ ]:
def FindAvgBranchingFactor(n):
    t = FastTreeExplore(n)
    d = t.avg_branching_factor()
    return d

df = pd.DataFrame({'_n': range(3,11), 'avgbranch':0, 'count':0, 'maxbranch':0, 'sumbranch':0})
for i in df.index:
    n = df['_n'][i]
    d = FindAvgBranchingFactor(n)
    df.iloc[i] = (n, d['avgbranch'], d['count'], d['maxbranch'], d['sumbranch'])
    
#df['avgbranch'] = df['n'].apply(FindAvgBranchingFactor)
branching_factor_df = df
branching_factor_df

In [ ]:
def FindAvgBranchingFactorByLevel(n):
    t = FastTreeExplore(n, trimming=False)
    d = t.avg_branching_factor_by_level()
    return d

def MakeAvgBranchingFactorByLevelTable(n):
    df = pd.DataFrame(index=range(n), columns=['avgbranch', 'count', 'maxbranch', 'sumbranch'])
    d = FindAvgBranchingFactorByLevel(n)
    for i, row in enumerate(d):
        df.iloc[i] = row
    return df[['avgbranch', 'maxbranch']].transpose()
    

In [ ]:
df = MakeAvgBranchingFactorByLevelTable(10)
df

In [ ]:
df = MakeAvgBranchingFactorByLevelTable(12)
df

In [15]:
DoCountMaxMoves(2)

2

In [ ]:
newdf = df[['avgbranch', 'maxbranch']].transpose()

print(newdf.to_latex())

In [16]:
df = pd.DataFrame({'n':range(1,11)})
#df['maxbranch'] = pd.Series([DoCountMaxMoves(n) for n in range(5,51,5)])

df['maxbranch'] = df['n'].apply(DoCountMaxMoves)
df

,n,maxbranch
0,1,1
1,2,2
2,3,3
3,4,6
4,5,16
5,6,36
6,7,78
7,8,150
8,9,272
9,10,474


In [ ]:
%%cython -a
import copy

def test():
    print("hello")


In [4]:
%%cython -a
import copy

def make_board(n=7):
    board = set(range(1,n+1))
    return board

cdef class FastGameNode:
    cdef int sums[3]
    cdef int movein_player, moveout_player
    cdef set board
    cdef int level
    
    def __init__(self, movein_player=2, board=make_board(), level=0):
        self.sums[0] = sum(board)
        self.sums[1] = 0
        self.sums[2] = 0    
        self.movein_player = movein_player
        self.moveout_player = 3-movein_player
        self.board = board
        self.level = level
                
    cdef void push_move(self, list move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level += 1
        cdef int m
        for m in move:
            self.board.discard(m)
            self.sums[self.movein_player] += m
            self.sums[0] -= m

    cdef void pop_move(self, list move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level -= 1
        cdef int m
        for m in move:
            self.board.add(m)
            self.sums[self.moveout_player] -= m
            self.sums[0] += m
        
    #cdef void make_moves_list(self, moves_out)
    def generate_move(self, maxplay, sum_move, move, picks, trimming):
        cdef int maxpick
        cdef int csum_move = sum_move
        cdef int cmaxplay = maxplay
        
        if move:
            maxpick = max(move)
        else:
            maxpick = 0
            
        cdef int len_move_plus_1 = len(move)+1
        cdef int len_picks = len(picks)
        cdef int pick

        for pick in picks:
            if pick < maxpick:
                # only emit the moves in increasing order 
                # (i.e don't emit both 2,3 and 3,2 if you are behind by 4, just emit 2,3)
                if trimming:
                    continue

            if pick in move:
                # can't pick something you've already picked
                continue

            move.append(pick)
            if len_move_plus_1 == len_picks:
                yield list(move)
            elif csum_move+pick >= cmaxplay:
                yield list(move)
            else:
                for m in self.generate_move(cmaxplay, csum_move+pick, move, picks, trimming):
                    yield m
            move.pop()
            
    def count_moves(self, maxplay, sum_move, move, picks, trimming):
        cdef int maxpick
        cdef int csum_move = sum_move
        cdef int cmaxplay = maxplay
        count = 0
        
        if move:
            maxpick = max(move)
        else:
            maxpick = 0
            
        cdef int len_move_plus_1 = len(move)+1
        cdef int len_picks = len(picks)
        cdef int pick

        for pick in picks:
            if pick < maxpick:
                # only emit the moves in increasing order 
                # (i.e don't emit both 2,3 and 3,2 if you are behind by 4, just emit 2,3)
                if trimming:
                    continue

            if pick in move:
                # can't pick something you've already picked
                continue

            move.append(pick)
            if len_move_plus_1 == len_picks:
                count += 1
            elif csum_move+pick >= cmaxplay:
                count += 1
            else:
                count += self.count_moves(cmaxplay, csum_move+pick, move, picks, trimming)
            move.pop()
        return count
    
    cdef str make_key(self, str extra=""):
        return "%d:%d/%d:%s:%s" % (self.movein_player, self.sums[1], self.sums[2], str(self.board), extra)
    
    cdef int scoreP1(self):
        cdef int sA = self.sums[1]
        cdef int sB = self.sums[2]
        return sA-sB
    
    cdef int win(self):
        cdef int s = self.score()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    cdef int winP1(self):
        cdef int s = self.scoreP1()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    cdef int score(self):
        cdef int sA = self.sums[self.movein_player]
        cdef int sB = self.sums[self.moveout_player]
        return sA-sB
    
    cdef int gameover(self):
        return self.sums[0] == 0
    
    cdef list possible_moves(self, int trimming):
        # A is the current player
        # B is the opponent player
        cdef int sA = self.sums[self.moveout_player]
        cdef int sB = self.sums[self.movein_player]

        cdef int maxplay = sB - sA
        #assert maxplay >= 0

        cdef list moves = list(self.generate_move(maxplay, 0, [], self.board, trimming))
        return moves
    
    
    def count_possible_moves(self, trimming=True):
        # A is the current player
        # B is the opponent player
        cdef int sA = self.sums[self.moveout_player]
        cdef int sB = self.sums[self.movein_player]

        cdef int maxplay = sB - sA
        #assert maxplay >= 0

        return self.count_moves(maxplay, 0, [], self.board, trimming)


cdef tuple _FastCountWinLoseDraw_explore(FastGameNode node, dict cache):
    cdef str key = node.make_key()
    cdef int[4] value
    cdef int[4] result

    # we've been here before, return the cached value
    if cache.has_key(key):
        return cache[key]

    result = [0,0,0,0]
    moves = node.possible_moves(False)
    if not moves:
        win = node.winP1()
        if win == 1:
            return (1,0,0,1)
        elif win == -1:
            return (0,1,0,1)
        elif win == 0:
            return (0,0,1,1)
    else:
        for move in moves:
            node.push_move(move)
            value = _FastCountWinLoseDraw_explore(node, cache)
            result[0] += value[0]
            result[1] += value[1]
            result[2] += value[2]
            result[3] += value[3]
            node.pop_move(move)
    t = tuple(result)
    cache[key] = t
    return t



def FastCountWinLoseDraw(n, firstmove=None):
    wld = {'WIN':0,'LOSE':0,'TIE':0,'TOTAL':0}
    cache = {}         
        
    node = FastGameNode(board=make_board(n))
    if firstmove:
        node.push_move(firstmove)
        
    result = _FastCountWinLoseDraw_explore(node, cache)
        
    wld['WIN'] = result[0]
    wld['LOSE'] = result[1]
    wld['TIE'] = result[2]
    wld['TOTAL'] = result[3]
    wld['WIN%']  = float(wld['WIN'])/wld['TOTAL']
    wld['LOSE%']  = float(wld['LOSE'])/wld['TOTAL']
    wld['TIE%']  = float(wld['TIE'])/wld['TOTAL']
        
    return wld
        
cdef int _FastMinDepth_explore(FastGameNode node, int depth, int maxdepth):
        #if depth >= maxdepth:
        #    return maxdepth+1

        if node.gameover():
            # found a game that ends at maxdepth, we are done
            return depth           
        elif depth+1 <= maxdepth:
            # haven't reached the maxdepth yet, keep looking
            moves = node.possible_moves(True)
            for move in moves:
                node.push_move(move)
                best = _FastMinDepth_explore(node, depth+1, maxdepth)
                node.pop_move(move)

                if best <= maxdepth:
                    return best
            # if here, we didn't find anything
            return maxdepth+1
        else:
            # fail case, truncating
            return maxdepth+1
        
cdef int FastMinDepth(int n=10):
    # use iterative deepening
    cdef FastGameNode root = FastGameNode(board=make_board(n))
    
    cdef int iterdepth
    cdef int v
    for iterdepth in xrange(1, n+1):
        v = _FastMinDepth_explore(root, 0, iterdepth)
        if v <= iterdepth:
            return v

    return -1

   
def DoAvgBranchingFactor(n=7):
    cdef FastGameNode root = FastGameNode(board=make_board(n))



def DoFastMinDepth(n=10):
    return FastMinDepth(n)

def DoCountMaxMoves(n=10):
    maxcount = 0
    cdef FastGameNode root

    for i in xrange(1,n+1):
        root = FastGameNode(board=make_board(n))
        
        # check first move
        count = root.count_possible_moves(trimming=False)
        maxcount = max(maxcount, count)
        
        # check second move
        root.push_move([i])
        count = root.count_possible_moves(trimming=False)
        maxcount = max(maxcount, count)
    
    return maxcount

ERROR: Cell magic `%%cython` not found.


In [172]:
%time FastCountWinLoseDraw(19)

CPU times: user 20h 23min 33s, sys: 2min 32s, total: 20h 26min 6s
Wall time: 1d 9h 13min 31s


{'LOSE': -1479704576,
 'LOSE%': -13.49581589958159,
 'TIE': -1225916416,
 'TIE%': -11.181111775254035,
 'TOTAL': 109641728,
 'WIN': -1479704576,
 'WIN%': -13.49581589958159}

In [171]:
%time FastTreeExplore(12, trimming=False).count_win_lose_draw_cached()

CPU times: user 7.63 s, sys: 22.5 ms, total: 7.65 s
Wall time: 7.68 s


{'LOSE': 203420160,
 'LOSE%': 0.42467532467532465,
 'TIE': 72161280,
 'TIE%': 0.15064935064935064,
 'TOTAL': 479001600,
 'WIN': 203420160,
 'WIN%': 0.42467532467532465}

In [ ]:
def MaxSelections(n):
    return np.floor(np.sqrt(2*n-7.0/4)+.5)

df = pd.DataFrame({'n':range(5,51,5)})
#df = pd.DataFrame({'n':range(1,10,1)})
#df['maxbranch'] = pd.Series([DoCountMaxMoves(n) for n in range(5,51,5)])

df['maxselect'] = df['n'].apply(MaxSelections)
df['maxbranch'] = df['n'].apply(DoCountMaxMoves)
#print(df.transpose().to_latex())
df

In [ ]:
def MaxSelections(n):
    return np.floor(np.sqrt(2*n-7.0/4)+.5)

df = pd.DataFrame({'n':range(3,21,1)})
#df = pd.DataFrame({'n':range(1,10,1)})
#df['maxbranch'] = pd.Series([DoCountMaxMoves(n) for n in range(5,51,5)])

df['maxselect'] = df['n'].apply(MaxSelections)
df['maxbranch'] = df['n'].apply(DoCountMaxMoves)
#print(df.transpose().to_latex())
df

In [ ]:
x = pd.Series(range(1,100))
y = np.sqrt(x)*np.log(x)
s = np.sqrt(x)
l = np.log(x)
df = pd.DataFrame({'x':x,'y':y,'s':s,'l':l})
df.plot()

In [8]:
%%cython -a
import copy

def make_board(n=7):
    board = set(range(1,n+1))
    return board

cdef class FastGameNode:
    cdef int sums[3]
    cdef int movein_player, moveout_player
    cdef set board
    cdef int level
    
    def __init__(self, movein_player=2, board=make_board(), level=0):
        self.sums[0] = sum(board)
        self.sums[1] = 0
        self.sums[2] = 0    
        self.movein_player = movein_player
        self.moveout_player = 3-movein_player
        self.board = board
        self.level = level
                
    cdef void push_move(self, list move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level += 1
        cdef int m
        for m in move:
            self.board.discard(m)
            self.sums[self.movein_player] += m
            self.sums[0] -= m

    cdef void pop_move(self, list move):
        # this is used for modifying a GameNode to a new state if you don't care about building a complete tree
        
        # swap in/out players
        self.movein_player, self.moveout_player = self.moveout_player, self.movein_player
        self.level -= 1
        cdef int m
        for m in move:
            self.board.add(m)
            self.sums[self.moveout_player] -= m
            self.sums[0] += m
        
    cdef str make_key(self, str extra=""):
        return "%d:%d/%d:%s:%s" % (self.movein_player, self.sums[1], self.sums[2], str(self.board), extra)

    cdef int scoreP1(self):
        cdef int sA = self.sums[1]
        cdef int sB = self.sums[2]
        return sA-sB
    
    cdef int win(self):
        cdef int s = self.score()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    cdef int winP1(self):
        cdef int s = self.scoreP1()
        if s < 0:
            return -1
        elif s > 0:
            return 1
        else:
            return 0
    
    cdef int score(self):
        cdef int sA = self.sums[self.movein_player]
        cdef int sB = self.sums[self.moveout_player]
        return sA-sB

    def generate_move(self, maxplay, sum_move, move, picks):
        cdef int maxpick
        cdef int csum_move = sum_move
        cdef int cmaxplay = maxplay
        
        if move:
            maxpick = max(move)
        else:
            maxpick = 0
            
        cdef int len_move_plus_1 = len(move)+1
        cdef int len_picks = len(picks)
        cdef int pick

        for pick in picks:
            if pick < maxpick:
                # only emit the moves in increasing order 
                # (i.e don't emit both 2,3 and 3,2 if you are behind by 4, just emit 2,3)
                continue

            if pick in move:
                # can't pick something you've already picked
                continue

            move.append(pick)
            if len_move_plus_1 == len_picks:
                yield list(move)
            elif csum_move+pick >= cmaxplay:
                yield list(move)
            else:
                for m in self.generate_move(cmaxplay, csum_move+pick, move, picks):
                    yield m
            move.pop()
    
    cdef int gameover(self):
        return self.sums[0] == 0
    
    def possible_moves(self):
        # A is the current player
        # B is the opponent player
        cdef int sA = self.sums[self.moveout_player]
        cdef int sB = self.sums[self.movein_player]

        cdef int maxplay = sB - sA
        #assert maxplay >= 0

        moves = list(self.generate_move(maxplay, 0, [], self.board))
        return moves
      
        
cdef int _FastMinDepth_explore(FastGameNode node, int depth, int maxdepth, dict cache):
    #if depth >= maxdepth:
    #    return maxdepth+1

    key = node.make_key(str(depth))
    if cache.has_key(key):
        return cache[key]
    
    if node.gameover():
        # found a game that ends at depth, we are done
        cache[key] = depth
        return depth           
    elif depth+1 <= maxdepth:
        # haven't reached the maxdepth yet, keep looking
        moves = node.possible_moves()
        for move in moves:
            node.push_move(move)
            best = _FastMinDepth_explore(node, depth+1, maxdepth, cache)
            node.pop_move(move)

            if best <= maxdepth:
                cache[key] = best
                return best
        # if here, we didn't find anything
        cache[key] = maxdepth+1
        return maxdepth+1
    else:
        # fail case, truncating
        cache[key] = maxdepth+1
        return maxdepth+1


def FastMinDepth(n=10, start=None, end=None):
    # use iterative deepening
    cdef FastGameNode root = FastGameNode(board=make_board(n))
    cdef dict cache = {}
    
    if start == None:
        start = 1
    if end == None:
        end = n

    cdef int iterdepth
    cdef int v
    for iterdepth in xrange(start, end+1):
        cache = {}
        v = _FastMinDepth_explore(root, 0, iterdepth, cache)
        if v <= iterdepth:
            return v

    return -1
    
cdef void _explore_FastUniqueStates(FastGameNode node, dict cache):
    cdef str key = node.make_key()
    if cache.has_key(key):
        return
    else:
        cache[key] = 1
        cache['TOTAL'] += 1

    cdef list moves = node.possible_moves(self.trimming)
    cdef list move
    if not moves:
        return 
    else:
        for move in moves:
            node.push_move(move)
            _explore_FastUniqueStates(node, cache)
            node.pop_move(move)

def FastUniqueStates(n=8):
    cdef dict cache = {'TOTAL':0}
    cdef FastGameNode root = FastGameNode(board=make_board(n))
   
    _explore_FastUniqueStates(root, cache)
        
    return cache['TOTAL']

cdef int _explore_FastNegamax(FastGameNode node, int depth, int color, dict cache):
    #node.show2()
    cdef str key = node.make_key()
    if cache.has_key(key):
        return cache[key]

    if depth == 0:
        cache[key] = color * node.winP1()
        return color * node.winP1()
    
    cdef list moves = node.possible_moves()
    if not moves:
        cache[key] = color * node.winP1()
        return color * node.winP1()

    cdef int bestValue = -100
    cdef int val
    
    for move in moves:
        node.push_move(move)
        val = -_explore_FastNegamax(node, depth-1, -color, cache)
        node.pop_move(move)
        bestValue = max( bestValue, val )
        # can't get any better than 1
        if bestValue == 1:
            break
    cache[key] = bestValue
    return bestValue


def FastNegamax(n=8):
    cdef FastGameNode root = FastGameNode(board=make_board(n))

    # negamax code ported from https://en.wikipedia.org/wiki/Negamax
    cache = {}
        
    return _explore_FastNegamax(root, n+1, 1, cache)


    

    


Error compiling Cython file:
------------------------------------------------------------
...
        return
    else:
        cache[key] = 1
        cache['TOTAL'] += 1

    cdef list moves = node.possible_moves(self.trimming)
                                             ^
------------------------------------------------------------

/Users/aisaksen/.ipython/cython/_cython_magic_66afbd745b5d95ba703872bcaaff89ac.pyx:187:46: undeclared name not builtin: self


In [ ]:
FastMinDepth(13)

In [ ]:
FastMinDepth(17, start=6, end=6)

In [ ]:
%timeit FastNegamax(10)
#%time FastTreeExplore(10).negamax_cached()

In [ ]:
t = FastTreeExplore(11)
%time a = t.mindepth_id()
%time b = FastMinDepth(11)
%time c = t.mindepth()
(a,b,c)

In [ ]:
t = FastTreeExplore(4)


In [ ]:
t = FastTreeExplore(14)
%time a = t.mindepth_id2()
a

In [ ]:
nmin = 3
nmax = 20
n = nmin
v = 1
mindepths = []
while n <= nmax:
    %time v = FastMinDepth(n, start = v)
    print(n, v)
    mindepths.append(v)
    n += 1
    
mindepths = pd.DataFrame({'n':range(nmin,nmax+1), 'mindepth':mindepths})
mindepths.index = range(nmin,nmax+1)
mindepths

In [12]:
class TreeBuilder:
    def __init__(self, n=4, build=False, trimming=True):
        self.root = GameNode(board=make_board(n))
        self.trimming = trimming
        self.next_vertexid = 0
        self.cache = {}
        if build == True:
            self.build()
            self.calc_results()
            
    def find_node(self, board, player, score):
        def _find(node):
            if board == node.board and node.moveout_player and node.score() == score:
                print(node.movein)
                return node

            # early exit if we are searching for something that has already been removed
            for needle in board:
                if needle not in node.board:
                    return None
                
            # search all children
            for child in node.children:
                found = _find(child)
                if found:
                    print(node.movein)
                    return found
                
            # nothing found
            return None
        
        return _find(self.root)
    
    def num_states(self):
        def _recurse(node):
            n = 1 # count 1 for the note
            for child in node.children:
                n += _recurse(child)
            return n
        return _recurse(self.root)
            
            
    def build(self, cache=True):
        def _step_cached(node):
            key = node.make_key()
            
            node.setmoves(self.trimming)
            for move in node.moves:
                child = node.make_move(move)
                key = child.make_key()
                if self.cache.has_key(key):
                    # we've already been down this path on a previous branch, so we can skip it
                    node.children.pop()
                    node.children.append(self.cache[key])
                else:
                    self.cache[key] = child
                    _step_cached(child)
                
        def _step_uncached(node):
            node.setmoves(self.trimming)
            for move in node.moves:
                node.make_move(move)

            for child in node.children:
                _step_uncached(child)
                                
        if cache:
            self.cache = {}
            _step_cached(self.root)
        else:
            _step_uncached(self.root)
        
    def make_graph(self, startnode=None, trimdepth=-1):
        g = igraph.Graph()
        
        if startnode == None:
            startnode = self.root
        
        self.next_vertexid = 0
        self.assign_vertexids(startnode)
        g.add_vertices(self.next_vertexid)
        
        edgelist, attrlist = self.assign_edges(startnode, [], [])
        g.add_edges(edgelist)
        
        self.setup_vertex_attributes(g, startnode)
        self.setup_edge_attributes(g, edgelist, attrlist)
                
        
        if trimdepth > 0:
            to_delete_ids = [v.index for v in g.vs if v['depth'] >= trimdepth]
            g.delete_vertices(to_delete_ids)
        return g
    
    def setup_vertex_attributes(self, g, node):
        v = g.vs[node.vertexid]
        
        v['size'] = 10
        
        if node.moveout_player == 1:
            v['shape'] = 'triangle-up'
        else:
            v['shape'] = 'square'

        board = [str(b) for b in node.board]
        text = '{' + ','.join(board) + '}:'+str(abs(node.scoreP1()))
        v['label'] = text
        #v['label'] = ""
        
        v['depth'] = node.level

        v['label_size'] = 9
        v['label_dist'] = 2
        v['label_angle'] = 3.14/2

        if -node.result == 0:
            v['color'] = '#CCCCCC'
        elif -node.result > 0:
            v['color'] = '#FFBF00'
        else:
            v['color'] = 'red'
            
        for child in node.children:
            self.setup_vertex_attributes(g, child)
    
    def setup_edge_attributes(self, g, edgelist, attrlist):
        i=0
        for attrs in attrlist:
            e = g.es[i]
            e['color'] = attrs[0]
            e['label'] = attrs[1]
            e['label_size'] = 9
            i+=1
            
    
    def assign_edges(self, node, inlist=[], attrlist=[]):
        if node.graphed:
            return inlist, attrlist
        
        node.graphed = True
        
        for i, child in enumerate(node.children):
            inlist.append((node.vertexid, child.vertexid))
                    
            if child.result == 0:
                color = '#888888'
            elif child.result < 0:
                color = 'red'
            else:
                color = '#e5ab00'
                
            move = [str(m) for m in node.moves[i]]
            label = ','.join(move)
            #label = ""
        
            attrlist.append((color, label))
                
        for child in node.children:
            self.assign_edges(child, inlist, attrlist)
        return inlist, attrlist
    
    def assign_vertexids(self, node):
        if node.vertexid == None:
            node.vertexid = self.next_vertexid
            self.next_vertexid += 1

            for child in node.children:
                self.assign_vertexids(child)
        
    def calc_results(self):
        self.calc_results_recursive(self.root)
    
    def calc_results_recursive(self, node):
        if node.result != None:
            # we've already calculated the result for this node (since we might have cyclic trees)
            return
        
        if not node.children:
            # terminal node
            node.result = node.win()
        else:
            # has children
            for child in node.children:
                self.calc_results_recursive(child)
            child_results = [child.result for child in node.children]
            node.result = -max(child_results)
           
    def calc_sg(self):
        def _explore(node):
            if node.sgvalue != None:
                # we've already calculated the sgvalue
                return
            if not node.children:
                node.sgvalue = 0

    def step(self, node):
        node.setmoves()
        for move in node.moves:
            node.make_move(move)
            
            

In [13]:
def Go(n=5, cache=True):
    t = TreeBuilder(n,trimming=False)
    %time t.build(cache=cache)
    %time t.calc_results()
    
Go(8)


CPU times: user 85 ms, sys: 13.4 ms, total: 98.4 ms
Wall time: 89.4 ms
CPU times: user 4.18 ms, sys: 2 µs, total: 4.18 ms
Wall time: 4.19 ms


## Game Tree Size

According to the wikipedia page on [Game complexity](https://en.wikipedia.org/wiki/Game_complexity), "The *game tree size* is the total number of possible games that can be played: the number of leaf nodes in the game tree rooted at the game's initial position."

In [ ]:
# this will be for the full expansion, not eliminating equivalent moves

n = 1
nmax = 20
nterminals = []
while n <= nmax:
    tree = FastTreeExplore(n, trimming=False)
    v = tree.num_terminals_cached()
    print(n, v)
    nterminals.append(v)
    n += 1
    
nterminals = pd.DataFrame({'n':range(1,nmax+1), 'treesize':nterminals})
nterminals.index = range(1,nmax+1)
nterminals

## State Space Complexity
this is the number of unique states that the game can be in.  we can ignore the moves that took us to the current state, and just look at the board selection, active player, and current score.

In [ ]:
%prun FastTreeExplore(12).unique_states()


In [ ]:
%time FastUniqueStates(13)

In [ ]:
n = 1
nmax = 20
nterminals = []
while n <= nmax:
    %time v = FastUniqueStates(n)
    print(n, v)
    nterminals.append(v)
    n += 1
    
nterminals = pd.DataFrame({'n':range(1,nmax+1), 'treesize':nterminals})
nterminals.index = range(1,nmax+1)
nterminals


## Game Depth
what are the minimum and maximum depths for a game with board $1...N$?

In [ ]:
t = FastTreeExplore(11)
#%prun t.mindepth_bfs()
#%time t.mindepth()


In [ ]:
nmax=10
trees = [FastTreeExplore(n) for n in xrange(1,nmax+1)]
s = pd.Series([tree.mindepth() for tree in trees])
s.index = range(1,nmax+1)
s

In [ ]:
def make_tree_image(filename, n, bbox, trimdepth=-1, treetype="tree", cache=False, trimming=True):
    t = TreeBuilder(n, trimming=trimming)
    t.build(cache=cache)
    t.calc_results()
    #t.root.show(recursive=True)
    g = t.make_graph(trimdepth=trimdepth)
    if treetype == 'tree' or treetype=='rt_circular':
        layout = g.layout(treetype,root=[0])
    else:
        layout = g.layout(treetype)
    igraph.plot(g, filename, bbox=bbox, layout=layout)

In [ ]:
def make_subtree_image(filename, n, bbox, board, player, scorediff, trimdepth=-1, treetype="tree", cache=False):
    t = TreeBuilder(n)
    t.build(cache=cache)
    t.calc_results()
    #t.root.show(recursive=True)
    node = t.find_node(board, player, scorediff)
    g = t.make_graph(startnode= node, trimdepth=trimdepth)
    if treetype == 'tree' or treetype=='rt_circular':
        layout = g.layout(treetype,root=[0])
    else:
        layout = g.layout(treetype)
    igraph.plot(g, filename, bbox=bbox, layout=layout)

In [ ]:
make_subtree_image("subtree-comeback.pdf", 7, (0,0,375,140), set([3,4,7]), 1, 2)

In [ ]:
make_subtree_image("subtree-dangerous.pdf", 7, (0,0,600,250), set([1,2,4,7]), 2, 4)

In [ ]:
make_subtree_image("subtree-dangerous2.pdf", 7, (0,0,600,275), set([1,3,4,6]), 2, 4)

In [ ]:
make_tree_image("cycle-tree-4.pdf", 4, (0,0,1000,1000), treetype = "auto", cache=True)

In [ ]:
make_tree_image("full-tree-4a.pdf", 4, (0,0,1000,300))
make_tree_image("full-tree-5a.pdf", 5, (0,0,2500,600))
make_tree_image("full-tree-6a.pdf", 6, (0,0,20000,600))
make_tree_image("full-tree-7a.pdf", 7, (0,0,64000,600))

In [ ]:
make_tree_image("paper/full-tree-4a.pdf", 4, (0,0,1000,250), trimming=False)

In [ ]:
make_tree_image("tree-7-trunc3.pdf", 7, (0,0,8000,8000), 8, "rt_circular")

In [ ]:
nmax=9
trees = [TreeBuilder(n, build=True) for n in xrange(1,nmax+1)]
nstates = pd.Series([tree.num_states() for tree in trees])
nstates.index = range(1,nmax+1)
nstates


In [ ]:
test = pd.DataFrame({'nstates':nstates})
test['nlogn'] = [n*np.log(n) for n in test.index]
test['lognstates'] = np.log(test['nstates'])
test.plot(x='nlogn', y='lognstates')
#test


## Optimal Play

For N, solve the optimal play using a cached negamax.

In [18]:
def SolveGame(n=8):
    t = TreeBuilder(n)
    t.build()
    t.calc_results()
    return -t.root.result

#%time print(FastNegamax(9))
%time FastTreeExplore(9).negamax_cached()

CPU times: user 140 ms, sys: 6.56 ms, total: 147 ms
Wall time: 142 ms


-1

In [22]:
t = TreeBuilder(9)
t.build()
t.calc_results()
for child in t.root.children:
    print(child.result)

-1
-1
-1
-1
-1
-1
-1
-1
-1


In [ ]:
ranges = range(3,20)
df = pd.DataFrame(columns=['n','winP1'])
df['n'] = ranges
df['winP1'] = pd.Series([FastNegamax(n) for n in ranges])
df

In [ ]:
g = GameNode(board=make_board(17))
g.push_move([17])

moves = g.possible_moves()
assert [7,9,16] in moves
g.push_move([7,9,16])

moves = g.possible_moves()
assert [6,8,15] in moves
g.push_move([6,8,15])

moves = g.possible_moves()
assert [3,10,14] in moves
g.push_move([3,10,14])

moves = g.possible_moves()
assert [1,11,13] in moves
g.push_move([1,11,13])

g.board